In [6]:
import pandas as pd
import psycopg2
import sqlalchemy

In [8]:
data_property = pd.read_csv('TR_PropertyInfo.csv')
data_user = pd.read_csv('TR_UserInfo.csv')

In [9]:
data_property.head()

,Prop ID,PropertyCity,PropertyState
0,1,New York,New York
1,2,Cincinnati,Ohio
2,3,Portland,Oregon
3,4,Seattle,Washington
4,5,Kansas City,Missouri


In [10]:
data_user.head()

,UserID,UserSex,UserDevice
0,1164,Male,Others
1,1003,Female,Others
2,1169,Male,Ios
3,1047,Female,Others
4,613,Female,Web


In [11]:
col_p= {"Prop ID": "prop_id" , "PropertyCity": "property_city", "PropertyState": "property_state"}
col_u= {"UserID": "user_id", "UserSex" :"user_id", "UserDevice": "user_device"}

In [12]:
data_property = data_property.rename(col_p)
data_user = data_user.rename(col_u)

In [14]:
#conn = sqlalchemy.create_engine(url='postgresql"//postgre:password@localhost:5432/postgres')
conn= psycopg2.connect(database= 'database', user ='postgres', password='mekanika', host='localhost', port='5432')
cursor=conn.cursor()

In [ ]:
data_property.to_sql('property', con=conn, index=False, if_exists='replace')
data_user.to_sql('user', con=conn, index=False, if_exists='replace')

In [16]:
class AggregateProduct(MRJob):

    def reducer_init(self):
        self.conn= psycopg2.connect(database= 'database', user ='postgres', password='mekanika', host='localhost', port='5432')

    def mapper(self, _, line):
        item=line.strip().split(',')
        year =item[1][-4:]
        yield item[1], int(item[4])

    def reducer(self, key, value):
        yield key, sum(values)

    def store(self,key,values):
        self.cur=self.conn.cursor()
        self.cur.execute("insert into total_order_yearly (year, total_order) values(%s, %s)",(key,values))

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                reducer=self.reducer
            )
            MRStep(
                mapper=self.mapper,
                reducer_init=self.reducer_init,
                reducer=self.reducer
            )
        ]
    def mapper_final(self):
        self.conn.commit()
        self.conn.close()

if __name__ == '__main__':
    AggregateProduct.run()

SyntaxError: invalid syntax (2124231379.py, line 24)